In [38]:
import org.apache.spark.sql.{DataFrame, Dataset, Row, SparkSession}
import org.apache.spark.sql.functions._
import scala.util.matching
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.types._

import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.Normalizer
import org.apache.spark.ml.evaluation.ClusteringEvaluator
import scalax.chart.api._
import scala.collection.mutable.Map

Unknown Error: <console>:58: error: not found: value scalax
       import scalax.chart.api._
              ^


In [2]:
val country = "US"
val usDF = spark.read.format("csv").option("header", "true")
                .load("data/" + country + "videos_new.csv")

country = US
usDF = [video_id: string, trending_date: string ... 14 more fields]


[video_id: string, trending_date: string ... 14 more fields]

In [3]:
usDF.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [4]:
val usDF1 = usDF.select($"category_id",$"comment_count",
                        $"dislikes",$"views",$"likes")
                        .na.drop()


usDF1.printSchema()

val usDF2 = usDF1.withColumn("category_id",col("category_id").cast(DoubleType))
    .withColumn("comment_count",col("comment_count").cast(IntegerType))
    .withColumn("dislikes",col("dislikes").cast(IntegerType))
    .withColumn("views",col("views").cast(IntegerType))
    .withColumn("likes",col("likes").cast(IntegerType))
usDF2.show(5)

root
 |-- category_id: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)

+-----------+-------------+--------+-------+------+
|category_id|comment_count|dislikes|  views| likes|
+-----------+-------------+--------+-------+------+
|       22.0|        15954|    2966| 748374| 57527|
|       24.0|        12703|    6146|2418783| 97185|
|       23.0|         8181|    5339|3191434|146033|
|       24.0|         2146|     666| 343168| 10172|
|       24.0|        17518|    1989|2095731|132235|
+-----------+-------------+--------+-------+------+
only showing top 5 rows



usDF1 = [category_id: string, comment_count: string ... 3 more fields]
usDF2 = [category_id: double, comment_count: int ... 3 more fields]


[category_id: double, comment_count: int ... 3 more fields]

In [5]:
val numNan = usDF.count - usDF1.count


48137

In [7]:
val assembler = new VectorAssembler()
                .setInputCols(Array("comment_count","dislikes",
                                    "views","category_id","likes"))
                .setOutputCol("features")
                .transform(usDF2)
usDF2.printSchema()

root
 |-- category_id: double (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)



assembler = [category_id: double, comment_count: int ... 4 more fields]


[category_id: double, comment_count: int ... 4 more fields]

In [8]:
assembler.select($"likes",$"features").show(5)

+------+--------------------+
| likes|            features|
+------+--------------------+
| 57527|[15954.0,2966.0,7...|
| 97185|[12703.0,6146.0,2...|
|146033|[8181.0,5339.0,31...|
| 10172|[2146.0,666.0,343...|
|132235|[17518.0,1989.0,2...|
+------+--------------------+
only showing top 5 rows



In [9]:
val normalizer= new Normalizer()
                .setInputCol("features")
                .setOutputCol("normfeatures")
                .setP(2.0)
                .transform(assembler)
normalizer.show(5)
normalizer.printSchema()

+-----------+-------------+--------+-------+------+--------------------+--------------------+
|category_id|comment_count|dislikes|  views| likes|            features|        normfeatures|
+-----------+-------------+--------+-------+------+--------------------+--------------------+
|       22.0|        15954|    2966| 748374| 57527|[15954.0,2966.0,7...|[0.02131320801961...|
|       24.0|        12703|    6146|2418783| 97185|[12703.0,6146.0,2...|[0.00525172527371...|
|       23.0|         8181|    5339|3191434|146033|[8181.0,5339.0,31...|[0.00256341245751...|
|       24.0|         2146|     666| 343168| 10172|[2146.0,666.0,343...|[0.00625336276642...|
|       24.0|        17518|    1989|2095731|132235|[17518.0,1989.0,2...|[0.00835860143342...|
+-----------+-------------+--------+-------+------+--------------------+--------------------+
only showing top 5 rows

root
 |-- category_id: double (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- dislikes: integer (nullable = 

normalizer = [category_id: double, comment_count: int ... 5 more fields]


[category_id: double, comment_count: int ... 5 more fields]

In [10]:
val Array(trainingData,testData)= normalizer.randomSplit(Array(0.7,0.3))

trainingData = [category_id: double, comment_count: int ... 5 more fields]
testData = [category_id: double, comment_count: int ... 5 more fields]


[category_id: double, comment_count: int ... 5 more fields]

In [44]:
val resultArr = scala.collection.mutable.Map[Int, Double]()
for( a <- 2 to 100){
    
    val kmeans = new KMeans().setK(a).setSeed(1L)
    val model = kmeans.fit(trainingData)
// Make predictions
    val predictions = model.transform(testData)
// Evaluate clustering by computing Silhouette score
    val evaluator = new ClusteringEvaluator()
    val silhouette = evaluator.evaluate(predictions)
    println(s"Silhouette with squared euclidean distance = $a,$silhouette")
    
    
    resultArr(a) = silhouette
    
}
println(resultArr)

Silhouette with squared euclidean distance = 2,0.9863755186210522
Silhouette with squared euclidean distance = 3,0.9620062403947562
Silhouette with squared euclidean distance = 4,0.9446297004458408
Silhouette with squared euclidean distance = 5,0.9426128629091324
Silhouette with squared euclidean distance = 6,0.9189003867808463
Silhouette with squared euclidean distance = 7,0.8771548044477305
Map(2 -> 0.9863755186210522, 5 -> 0.9426128629091324, 4 -> 0.9446297004458408, 7 -> 0.8771548044477305, 3 -> 0.9620062403947562, 6 -> 0.9189003867808463)


arr1 = Map(2 -> 0.9863755186210522, 5 -> 0.9426128629091324, 4 -> 0.9446297004458408, 7 -> 0.8771548044477305, 3 -> 0.9620062403947562, 6 -> 0.9189003867808463)


Map(2 -> 0.9863755186210522, 5 -> 0.9426128629091324, 4 -> 0.9446297004458408, 7 -> 0.8771548044477305, 3 -> 0.9620062403947562, 6 -> 0.9189003867808463)

In [ ]:
// Save Array to file
val resultDF = spark.sparkContext
        .parallelize(resultArr.toSeq.sortBy(_._1))
        .toDF("num_clusters", "silhouette")


resultDF.repartition(1)
        .write
        .format("csv")
        .option("header", "true")
        .save("result_clustering")